In [2]:
#앙상블 모델  //Boosting

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.model_selection import *
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree

In [4]:
plt.rcParams['figure.figsize'] = [7, 7]
sns.set(style='darkgrid')
plt.rcParams['scatter.edgecolors'] = 'black'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)
pd.set_option('display.unicode.east_asian_width', True)

In [5]:
iris_dataset = load_iris()
iris = pd.DataFrame(iris_dataset.data,
        columns=iris_dataset.feature_names)
labels = iris_dataset.target_names
iris.info()
print(iris.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
dtypes: float64(4)
memory usage: 4.8 KB
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
1                4.9               3.0                1.4               0.2
2                4.7               3.2                1.3               0.2
3                4.6               3.1                1.5               0.2
4                5.0               3.6                1.4               0.2


In [6]:
label = iris_dataset.target
print(label)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [7]:
scaler = StandardScaler()
iris = scaler.fit_transform(iris)
Features = pd.DataFrame(iris, columns=['SL', 'SW', 'PL', 'PW'])
print(Features.shape)

(150, 4)


In [8]:
#데이터 분할
X_train, X_test, Y_train, Y_test = train_test_split(
            Features, label, test_size=0.2)

In [ ]:
#AdaBoosting : 이전의 분류기가 잘못 분류햔 샘플에 대한 가중치를 높여서 다음 모델을 훈련시킴, 반복마다 샘플에 대한 가중치를 수정
#              훈련된 각 분류기는 성능에 따라 가중치가 부여되고, 예측에서 각 분류기가 예측한 레이블을 기준으로 가중치까지 적용해 가장 높은 값을 가진 레이블을 선택
#AdaBoostClassifier()
#base_estimator : 앙상블에 포함될 기본 분류기 종류, 기본값으로 DecisionTreeClassifier(max_depth=1) 을 사용
# n_estimators : 모델에 포함될 분류기 개수
# learnig_rate : 학습률
# algorithm : 부스팅 알고리즘, 기본값='SAMME.R'
# random_state : 난수 seed 설정


In [9]:
n_estimators = [50, 100, 150, 200, 250, 300, 350, 400]
learning_rate = [0.1, 0.2, .3, .4, .5, .6, .7, .8, .9, 1.0]
algorithm = ['SAMME', 'SAMME.R']
param = {'n_estimators':n_estimators, 'learning_rate':learning_rate,
         'algorithm':algorithm}
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, 
                            random_state=868)

#n_estimators 와 learning_rate에 따른 최적 모델....?
iris_AdaB = GridSearchCV(estimator=AdaBoostClassifier(),
            param_grid=param, scoring='accuracy',
            n_jobs=-1, cv=cv)
iris_AdaB.fit(X_train, Y_train)

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=868, test_size=0.2,
            train_size=None),
             estimator=AdaBoostClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,
                                           0.8, 0.9, 1.0],
                         'n_estimators': [50, 100, 150, 200, 250, 300, 350,
                                          400]},
             scoring='accuracy')

In [10]:
print(iris_AdaB.best_score_)
print(iris_AdaB.best_params_)
print(iris_AdaB.best_estimator_)

0.9666666666666668
{'algorithm': 'SAMME.R', 'learning_rate': 1.0, 'n_estimators': 100}
AdaBoostClassifier(n_estimators=100)


In [15]:
for i in range(1, 100):
    X_train, X_test, Y_train, Y_test = train_test_split(
            Features, label, test_size=0.2, random_state=i)
    iris_AdaB = AdaBoostClassifier(n_estimators=100,
                    learning_rate=1.0, algorithm='SAMME.R')
    iris_AdaB.fit(X_train, Y_train)

    train_score = iris_AdaB.score(X_train, Y_train)
    test_score = iris_AdaB.score(X_test, Y_test)
    if test_score >= train_score:
        print('test:{} train:{} random_state:{}'.format(
            test_score, train_score, i))

test:0.9666666666666667 train:0.9416666666666667 random_state:1
test:0.9333333333333333 train:0.9333333333333333 random_state:3
test:1.0 train:0.9666666666666667 random_state:9
test:1.0 train:0.95 random_state:10
test:0.9666666666666667 train:0.9583333333333334 random_state:12
test:0.9666666666666667 train:0.9583333333333334 random_state:13
test:0.9666666666666667 train:0.9416666666666667 random_state:14
test:0.9333333333333333 train:0.9333333333333333 random_state:15
test:1.0 train:0.9666666666666667 random_state:17
test:1.0 train:0.9666666666666667 random_state:18
test:1.0 train:0.9666666666666667 random_state:19
test:0.9666666666666667 train:0.9583333333333334 random_state:23
test:0.9666666666666667 train:0.9166666666666666 random_state:24
test:0.9666666666666667 train:0.9416666666666667 random_state:26
test:1.0 train:0.9666666666666667 random_state:35
test:0.9666666666666667 train:0.9416666666666667 random_state:36
test:1.0 train:0.95 random_state:37
test:0.9333333333333333 train:0

In [11]:
for i in range(1, 100):
    X_train, X_test, Y_train, Y_test = train_test_split(
            Features, label, test_size=0.2, random_state=i)
    iris_AdaB = AdaBoostClassifier(n_estimators=350,
                    learning_rate=0.1, algorithm='SAMME')
    iris_AdaB.fit(X_train, Y_train)

    train_score = iris_AdaB.score(X_train, Y_train)
    test_score = iris_AdaB.score(X_test, Y_test)
    if test_score >= train_score:
        print('test:{} train:{} random_state:{}'.format(
            test_score, train_score, i))

test:1.0 train:0.9833333333333333 random_state:9
test:1.0 train:0.9833333333333333 random_state:15
test:1.0 train:0.9833333333333333 random_state:18
test:1.0 train:0.9833333333333333 random_state:19
test:1.0 train:0.9833333333333333 random_state:40
test:1.0 train:0.9916666666666667 random_state:42
test:1.0 train:0.9833333333333333 random_state:70
test:1.0 train:0.9833333333333333 random_state:72
test:1.0 train:0.9833333333333333 random_state:84
test:1.0 train:0.9833333333333333 random_state:91


In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(
            Features, label, test_size=0.2, random_state=42)
iris_AdaB = AdaBoostClassifier(n_estimators=350,
                learning_rate=0.1, algorithm='SAMME')
iris_AdaB.fit(X_train, Y_train)

train_score = iris_AdaB.score(X_train, Y_train)
test_score = iris_AdaB.score(X_test, Y_test)

print('test:{} train:{}'.format(
        test_score, train_score))

test:1.0 train:0.9916666666666667


In [13]:
pd.DataFrame(confusion_matrix(Y_test, iris_AdaB.predict(X_test)),
        columns=['P_setosa', 'P_versicolor', 'P_virginica'],
        index=['A_setosa', 'A_versicolor', 'A_virginica'])

,P_setosa,P_versicolor,P_virginica
A_setosa,10,0,0
A_versicolor,0,9,0
A_virginica,0,0,11


In [14]:
print(classification_report(Y_test, iris_AdaB.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

